In [ ]:
### permitted modules ###
import numpy as np
import datetime
import hashlib
#########################

### comments about code and algorithms used (optional) ###

##########################################################

In [15]:
###  utility constants, functions and lookups can be instantiated/built here ###

def get_stats(raw):

	
    
	lines = raw.splitlines()

	wins = []

	lines = lines[1:1001]

	dealers = []

	stats = []
	tips = []

	#line = "1,TRUE,FALSE,FALSE,TRUE,FALSE,TRUE,FALSE,TRUE,FALSE,TRUE,FALSE,FALSE,TRUE,FALSE,1970 January 01 15:01:57.000,Homer,0,5,2500,0,2500,500.0"
	#arr = line.split(',')
	#dealer = arr[16]

	#a = check_dealer(dealer, dealers, stats)
	#get_dealer_stats(dealer, line, dealers, stats)


	#line = "2,TRUE,FALSE,FALSE,TRUE,TRUE,FALSE,TRUE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,FALSE,1970 January 01 15:04:19.000,Homer,0,6,14400,0,14400,2400.0"
	#arr = line.split(',')
	#dealer = arr[16]

	#a = check_dealer(dealer, dealers, stats)
	#get_dealer_stats(dealer, line, dealers, stats)

	#print(stats)

	#return

	# Read lines 2 to 101
	for line_number, line in enumerate(lines):
		#print(line, end='')
		


		#line = "1,TRUE,FALSE,FALSE,TRUE,FALSE,TRUE,FALSE,TRUE,FALSE,TRUE,FALSE,FALSE,TRUE,FALSE,1970 January 01 15:01:57.000,Homer,0,5,2500,0,2500,500.0"

		arr = line.split(',')


		#print(arr)

		# Color
		dealer = arr[16]
		tip = int(arr[17])

		check_dealer(dealer, dealers, stats, tips)
		#numbers
		get_dealer_stats(dealer, line, dealers, stats)
		#tips
		get_tips(dealer, dealers, tip, tips)


	print(stats)
	print(tips)
		


def check_dealer(dealer, dealers, stats, tips):
	if dealer not in dealers:
		dealers.append(dealer)
		stats.append([0 for i in range(14)])
		tips.append(0)
		return False
	else:
		return True
	
def get_dealer_stats(dealer, line, dealers, stats):
	index = dealers.index(dealer)
	arr = line.split(',')
	arr = arr[1::]
	for i in range(14):
		stats[index][i] += eval(arr[i].capitalize())

def get_tips(dealer, dealers, tip, tips):
	index = dealers.index(dealer)
	tips[index] += tip
################################################################################

def read_file_content(filename):
    f = open(filename, 'r', encoding = 'UTF-8')
    content = f.read()
    f.close()
    return content

raw = read_file_content("data_raw_1.txt")

get_stats(raw)

[[162, 171, 163, 170, 166, 167, 104, 124, 105, 104, 105, 124, 104, 89], [96, 92, 83, 105, 81, 107, 60, 57, 71, 63, 67, 58, 65, 52], [131, 146, 146, 131, 145, 132, 92, 97, 88, 94, 92, 91, 85, 84], [66, 75, 66, 75, 61, 80, 42, 48, 51, 47, 47, 47, 44, 36]]
[22300, 10800, 19100, 5200]


In [ ]:
### algorithm to compute analytics ###
def add_analytics_to_raw_data(raw):
    ### add logic here
    processed = raw
    ###
    return processed ### raw data followed by analytics section (should be the same as the data_analytics_memo_{number}.txt)
######################################

In [13]:
####### !!! DO NOT MODIFY !!! #######
def read_file_content(filename):
    f = open(filename, 'r', encoding = 'UTF-8')
    content = f.read()
    f.close()
    return content

def write_file_content(filename, content):
    f = open(filename, 'w', encoding = 'UTF-8')
    f.write(content)
    f.close()
    return None

def do_analyse_data(data_raw, write_file_name):
    data_analytics = add_analytics_to_raw_data(data_raw)
    write_file_content(write_file_name, data_analytics)
    return data_analytics

def analyse_data(file_number = '1'):
    base_filename = 'data_'
    
    ### read raw file and analytics memo file content from files with specified number
    data_raw = read_file_content(base_filename + 'raw_' + str(file_number) + '.txt')
    data_analytics_memo = read_file_content(base_filename + 'analytics_memo_' + str(file_number) + '.txt')
    
    ### generate analytics from raw file
    data_analytics = do_analyse_data(data_raw, base_filename + 'analytics_' + str(file_number) + '.txt')
    
    return data_raw, data_analytics, data_analytics_memo

def evaluate_performance(file_number = '1'):
    raw, analytics, analytics_memo = analyse_data(file_number)
    
    # ### print size and type
    # print(raw, type(raw))
    # print(analytics, type(analytics))
    # print(analytics_memo, type(analytics_memo))
    
    analytics_hash = hashlib.md5(analytics.encode()).hexdigest()
    analytics_memo_hash = hashlib.md5(analytics_memo.encode()).hexdigest()
    
    is_equal = analytics_hash == analytics_memo_hash ## md5 checksum
    
    return is_equal
####### !!! DO NOT MODIFY !!! #######

### PERFORMANCE
REQUIREMENT: Generated output file should be identical to the memo file.

In [ ]:
### replace file_number with the number of the txt file you want to test on
file_number = '1'

### run cell to get your score
is_requirement_satisfied = evaluate_performance(file_number)

print('Requirement satisfied: ' + str(is_requirement_satisfied))